## <center>Рекомендательные системы<center>

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings("ignore",category=UserWarning)

### Загрузка обучающей выборки

Разделение выборки по неделям на:
 - обучающей выборку для первого уровня двухуровневой модели;
 - обучающей выборку для второго уровня;
 - валидационную выборку.

In [2]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


### Предобработка обучающей выборки для первого уровня

Наилучший результат дает предварительная фильтрация топ-500 товаров:

In [3]:
from src.utils import filter_top_items

In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = filter_top_items(data_train_lvl_1.copy(), take_n_popular=500)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 501


### Датафреймы для результатов

Пользователи из обучающей выборки для второго уровня:

In [5]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


Пользователи из валидационной выборки:

In [6]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']
result_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### ALS-модель

In [7]:
from src.metrics import precision_at_k, recall_at_k
from src.recommenders import MainRecommender

Наилучший результат дает ALS-модель с *tfidf*-взвешиванием *user-item* матрицы:

In [8]:
np.random.seed(42)

als_recs = MainRecommender(data_train_lvl_1, 'tfidf')

Preparing tops...
Preparing matrix...
Preparing dicts...
Weighting...
Fitting als...


100%|████████████████████████████████████████| 15.0/15 [00:03<00:00,  4.28it/s]


Fitting own recommender...


100%|██████████████████████████████████████| 501/501 [00:00<00:00, 4911.49it/s]


Complete.


Добавим в датафреймы с результатами топ самых популяных товаров каждому пользователю:

In [9]:
result_lvl_1['top'] = result_lvl_1['user_id'].apply(lambda x: als_recs.get_top_items(x, N=200))
result_lvl_2['top'] = result_lvl_2['user_id'].apply(lambda x: als_recs.get_top_items(x, N=5))

In [ ]:
Рассчитаем для них 

In [10]:
print('Recall@2000')
print('top recommendations: {:.2f}%'.format(
       result_lvl_1.apply(lambda row: recall_at_k(row['top'], row['actual'], k=200), axis=1).mean() * 100), end='\n\n')

print('Precision@5')
print('top recommendations: {:.2f}%'.format(
       result_lvl_2.apply(lambda row: precision_at_k(row['top'], row['actual'], k=5), axis=1).mean() * 100))

Recall@2000
top recommendations: 7.62%

Precision@5
top recommendations: 5.89%


In [11]:
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: als_recs.get_own_recommendations(x, N=200))
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: als_recs.get_als_recommendations(x, N=200))

In [12]:
print('Recall@2000')
for recs in ['als', 'own']:
    print('{} recommendations: {:.2f}%'.format(recs, 
           result_lvl_1.apply(lambda row: recall_at_k(row[recs], row['actual'], k=200), axis=1).mean() * 100))

Recall@2000
als recommendations: 19.64%
own recommendations: 16.94%


выше поэтому

Добавим рекомендации ALS-модели в результаты с пользователями из обучающей выборки для второго уровня:

In [13]:
result_lvl_2['als'] = result_lvl_2['user_id'].apply(lambda x: als_recs.get_als_recommendations(x, N=5))

### Двухуровневая модель

Первый уровень:
 - ALS-модель с *tfidf*-взвешиванием *user-item* матрицы.

Второй уровень:
 - на основе бинарного классификатора LGBMClassifier;
 - классификатор обучается на лучше всего показавших себя по *recall@2000* рекомендациях ALS-модели;
 - в качестве признаков добавляются данные о товарах, пользователях, количество транзакций каждого пользователя (*n_transactions*), количество посещенных им магазинов (*user_per_stores*), количества посещенных им отделов (*n_departments*), его средний чек (*average_bill*).

In [14]:
from src.recommenders import LightGBRecommender

Обучение модели:

In [15]:
np.random.seed(42)

lgb_recs = LightGBRecommender(data_train_lvl_1, data_train_lvl_2, item_features, user_features)

FIRST LEVEL:
Preparing tops...
Preparing matrix...
Preparing dicts...
Weighting...
Fitting als...


100%|████████████████████████████████████████| 15.0/15 [00:03<00:00,  4.29it/s]


Fitting own recommender...


100%|██████████████████████████████████████| 501/501 [00:00<00:00, 3604.11it/s]


Complete.

SECOND LEVEL:
Preparing users and items...
Merging features...
Fitting lgb...
Complete.


Важность признаков:

In [16]:
lgb_recs.feature_importances_

sub_commodity_desc      1061
curr_size_of_product     668
manufacturer             515
commodity_desc           339
average_bill             115
n_departments            107
n_transactions            78
department                33
hh_comp_desc              17
income_desc               17
user_per_stores           14
brand                     14
age_desc                   7
marital_status_code        6
kid_category_desc          5
household_size_desc        4
homeowner_desc             0
dtype: int32

### Работа двухуровневой модели на валидационной выборке

Загрузим пользователей из валидационной выборки и пользователей из обучающей выборки для второго уровня:

In [17]:
valid_users = data_val_lvl_2['user_id'].unique()
train_users = data_train_lvl_2['user_id'].unique()

Получим для пользователей из валидационной выборки топ-5 рекомендаций двухуровневневой модели:

In [18]:
answers = lgb_recs.predict(valid_users, train_users, item_features, user_features)

answers.head()

,user_id,lgb
0,1,"[1082185, 914190, 1029743, 1070820, 1106523, 1..."
1,6,"[1082185, 1029743, 1070820, 1106523, 1126899, ..."
2,7,"[1082185, 1029743, 1070820, 1106523, 1126899, ..."
3,8,"[1082185, 1029743, 1070820, 1106523, 1126899, ..."
4,9,"[1082185, 1029743, 1070820, 1106523, 1126899, ..."


Добавим в результаты:

In [19]:
result_lvl_2 = result_lvl_2.merge(answers, on='user_id', how='left')

result_lvl_2.head()

,user_id,actual,top,als,lgb
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120741, 6533889, 1053690, 894236, 6534178]","[1005186, 1033142, 986912, 1110843, 1082185]","[1082185, 914190, 1029743, 1070820, 1106523, 1..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1120741, 6533889, 1053690, 894236, 6534178]","[951590, 5568378, 1106523, 910032, 5569327]",NaN
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1120741, 6533889, 1053690, 894236, 6534178]","[878996, 866227, 1127831, 5569230, 1023720]","[1082185, 1029743, 1070820, 1106523, 1126899, ..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1120741, 6533889, 1053690, 894236, 6534178]","[961554, 1082185, 938700, 1127831, 883404]","[1082185, 1029743, 1070820, 1106523, 1126899, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1120741, 6533889, 1053690, 894236, 6534178]","[1053690, 1029743, 1005186, 995242, 995785]","[1082185, 1029743, 1070820, 1106523, 1126899, ..."


Заполним пропущенных пользоватей рекомендяциями ALS-модели:

In [20]:
result_lvl_2['lgb'] = np.where(result_lvl_2['lgb'].isna(),
                               result_lvl_2['als'],
                               result_lvl_2['lgb'])

### Собственные покупки

Наилучший результат дает модель, рекомендующая собственные покупки с *bm25*-взвешиванием *user-item* матрицы:

In [21]:
np.random.seed(42)

own_recs = MainRecommender(data_train_lvl_1, 'bm25')

Preparing tops...
Preparing matrix...
Preparing dicts...
Weighting...
Fitting als...


100%|████████████████████████████████████████| 15.0/15 [00:03<00:00,  4.94it/s]


Fitting own recommender...


100%|██████████████████████████████████████| 501/501 [00:00<00:00, 6109.40it/s]


Complete.


Добавим для пользователей из валидационной выборки топ-5 рекомендаций собственных покупок:

In [22]:
result_lvl_2['own'] = result_lvl_2['user_id'].apply(lambda x: own_recs.get_own_recommendations(x, N=5))

### *precision@5* на результатах из валидационной выборки

In [23]:
print('Precision@5:')
for recs in ['lgb', 'als', 'own']:
    print('{} recommendations: {:.2f}%'.format(recs, 
           result_lvl_2.apply(lambda row: precision_at_k(row[recs], row['actual'], k=5), axis=1).mean() * 100))

Precision@5:
lgb recommendations: 18.69%
als recommendations: 19.72%
own recommendations: 29.28%


По значениям метрики *precision@5* видно, что удалось достичь хорошего качества, но есть большой отрыв у модели, рекомендующей собственные покупки.

### Работа лучшей модели на тестовой выборке

Загружаем пользователей из тестового датафрейма:

In [24]:
data_test = pd.read_csv('./data/retail_test1.csv')
test_users = data_test['user_id'].unique()

Получим для каждого из них топ-5 рекомендаций собственных покупок и сохраним результат в файл `recommendations.csv`:

In [25]:
result_test = pd.DataFrame({'user_id': test_users})
result_test['recs'] = result_test['user_id'].apply(lambda x: own_recs.get_own_recommendations(x, N=5))

result_test.head()

,user_id,recs
0,1340,"[951412, 849843, 874736, 821083, 861272]"
1,588,"[1070015, 951590, 1050851, 1053690, 933835]"
2,2070,"[1085604, 1055863, 1080414, 878302, 949616]"
3,1602,"[1058997, 1070820, 1095275, 860776, 862349]"
4,447,"[1057260, 937626, 926905, 999971, 849843]"


In [26]:
result_test.to_csv('recommendations.csv')